In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

%matplotlib inline

In [4]:
HouseDF = pd.read_csv('./USA_Housing.csv')
HouseDF.dropna(inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: './USA_Housing.csv'

In [ ]:
x = HouseDF[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms', 'Area Population']]
y = HouseDF['Price']  # Extracting target variable directly as a Series

print("Số lượng dữ liệu sau khi xử lý bác bỏ:", len(HouseDF))

In [ ]:
HouseDF.info()
HouseDF.describe()

In [ ]:
# Phân tích đa biến
sns.pairplot(HouseDF)
plt.show()

In [ ]:
# Phân tích đơn biến
plt.figure(figsize=(12, 8))
for i, column in enumerate(x.columns, 1):
    plt.subplot(2, 3, i)
    sns.histplot(x[column], kde=True)
    plt.title(f'Distribution of {column}')
plt.tight_layout()
plt.show()

In [ ]:
# Phân tích tương quan.
plt.figure(figsize=(10, 8))
sns.heatmap(HouseDF.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
sns.histplot(HouseDF['Price'])

In [ ]:
plt.figure(figsize=(15, 10))
for i, column in enumerate(x.columns, 1):
    plt.subplot(2, 3, i)
    sns.regplot(x=x[column], y=y, scatter_kws={'s': 10})
    plt.title(f'Reg-plot for {column} vs Price')

plt.tight_layout()
plt.show()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=101)


In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


In [ ]:
# Linear Regression
lm = LinearRegression()
lm.fit(x_train, y_train)
lm_predictions = lm.predict(x_test)
plt.scatter(y_test,lm_predictions)
lm_coefficients = lm.coef_

print("Linear Regression Metrics:")
print('MAE:', metrics.mean_absolute_error(y_test, lm_predictions))
print('MSE:', metrics.mean_squared_error(y_test, lm_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, lm_predictions)))
print("Linear Regression Metrics:")
print('Coefficients:')
for i, column in enumerate(x.columns):
    print(f'{column}: {lm_coefficients[i]}')

In [ ]:
# Decision Tree
dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(x_train, y_train)
dt_predictions = dt_regressor.predict(x_test)
plt.scatter(y_test,dt_predictions)
dt_feature_importances = dt_regressor.feature_importances_

print("\nDecision Tree Metrics:")
print('MAE:', metrics.mean_absolute_error(y_test, dt_predictions))
print('MSE:', metrics.mean_squared_error(y_test, dt_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, dt_predictions)))
print('Feature Importances:')
for i, column in enumerate(x.columns):
    print(f'{column}: {dt_feature_importances[i]}')

In [ ]:
# Random Forest
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(x_train, y_train)
rf_predictions = rf_regressor.predict(x_test)
plt.scatter(y_test,rf_predictions)
rf_feature_importances = rf_regressor.feature_importances_

print("\nRandom Forest Metrics:")
print('MAE:', metrics.mean_absolute_error(y_test, rf_predictions))
print('MSE:', metrics.mean_squared_error(y_test, rf_predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, rf_predictions)))
print('Feature Importances:')
for i, column in enumerate(x.columns):
    print(f'{column}: {rf_feature_importances[i]}')


In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

# Pipeline for preprocessing numerical features
numeric_features = ['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms', 'Area Population']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Pipeline for preprocessing categorical features if any
categorical_features = []  # Add categorical features if any
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessor combining the numeric and categorical transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Append the preprocessor to the model pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', RandomForestRegressor())])

# Define parameters grid for tuning
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 5, 10, 15],
    # Add more parameters to search if necessary
}

# Perform Grid Search Cross Validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

# Get the best model from Grid Search CV
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_model.predict(x_test)
print('Best Model Evaluation:')
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

# Get feature importances from the best model
feature_importances = best_model.named_steps['model'].feature_importances_
for i, column in enumerate(x.columns):
    print(f'{column}: {feature_importances[i]}')
